In [198]:
import numpy as np
from dotenv import load_dotenv
from dwave.system import DWaveSampler, EmbeddingComposite
from numpy import hamming
from openjij import SQASampler
import openjij
from pyqubo import Array, Constraint, LogEncInteger, solve_qubo
import math
import os
import dimod
from minorminer import find_embedding
from dwave.system import FixedEmbeddingComposite
from dwave.embedding import embed_bqm
from dwave.embedding import MinimizeEnergy
from matplotlib import pyplot as plt
import sklearn
import itertools

In [199]:
load_dotenv()

TOKEN = os.getenv("TOKEN")
endpoint = "https://cloud.dwavesys.com/sapi/"

In [200]:
W = 750
cost_list = [135,
        139,
        149,
        150,
        156,
        163,
        173,
        184,
        192,
        201,
        210,
        214,
        221,
        229,
        240, ]
weight_list = [70,
          73,
          77,
          80,
          82,
          87,
          90,
          94,
          98,
          106,
          110,
          113,
          115,
          118,
          120, ]
opt = [1,
       0,
       1,
       0,
       1,
       0,
       1,
       1,
       1,
       0,
       0,
       0,
       0,
       1,
       1]
N = len(cost_list)

In [201]:
x = Array.create('x', shape=(N), vartype='BINARY')
y = LogEncInteger("y", (0, W))

In [202]:
# key1 = max(cost_list, key=lambda k: cost_list[k])
max_c = max(cost_list)
B = 1
A = B * max_c + 1

In [203]:
HA = Constraint(
    A * (W - sum(weight_list[a] * x[a] for a in range(N)) - y)**2, label='HA'
)

HB = - B * sum(cost_list[a] * x[a] for a in range(N))

In [204]:
H = HA + HB
model = H.compile()
Q, offset = model.to_qubo()

In [205]:
dw_sampler = DWaveSampler(solver='DW_2000Q_6',
                          token=TOKEN, endpoint=endpoint)
adj = {}
for k in Q.keys():
    adj[k] = 1
embedding = find_embedding(adj, dw_sampler.edgelist)
# bqm_embed = embed_bqm(bqm, embedding, dw_sampler.adjacency)
sampler = FixedEmbeddingComposite(dw_sampler, embedding)
# sampler = EmbeddingComposite(dw_sampler)
# sampleset_embed = dw_sampler.sample(bqm_embed, num_reads=10)

In [231]:
t = [(0, 20), (0, 5, 10, 20), (0, 5, 20), (0, 10, 15, 20)]
s = [(0, 1), (0, 0.5, 0.5, 1), (0, 0, 1), (0, 0.75, 0.75, 1)]

schedules = [list(zip(t[i], s[i])) for i in range(len(t))]
print(schedules)

[[(0, 0), (20, 1)], [(0, 0), (5, 0.5), (10, 0.5), (20, 1)], [(0, 0), (5, 0), (20, 1)], [(0, 0), (10, 0.75), (15, 0.75), (20, 1)]]


In [215]:
samplesets = []
for schedule in schedules:
    samplesets.append(sampler.sample_qubo(
        Q, num_reads=1, anneal_schedule=schedule))

In [223]:
def transform_sample(sampleset, model):
    decoded_sample = model.decode_sample(
        sampleset.first.sample, vartype="BINARY")

    cost = 0
    weight = 0
    for k in range(N):
        if decoded_sample.array('x', k) != 0:
            cost += cost_list[k]
            weight += weight_list[k]

    sol_y = sum(2**k * v for k, v in [(elem, decoded_sample.array('y', elem))
                                      for elem in range(math.ceil(math.log2(W)))])

    return {"slack_y": sol_y, "broken": decoded_sample.constraints(only_broken=True), "cost": cost, "weight": weight}

v_transform_sample = np.vectorize(transform_sample)

In [226]:
def output_sample_info(sample_info):
    print("slack_Y = {}".format(sample_info["slack_y"]))
    print("broken: {}".format(sample_info["broken"]))
    print("cost : "+str(sample_info["cost"]))
    print("weight : "+str(sample_info["weight"]))
    print()

v_output_sample_info = np.vectorize(output_sample_info)

In [227]:
sample_info = v_transform_sample(samplesets, model)
v_output_sample_info(sample_info)

slack_Y = 327
broken: {'HA': (False, 13766161.0)}
cost : 1267
weight : 662

slack_Y = 327
broken: {'HA': (False, 13766161.0)}
cost : 1267
weight : 662

slack_Y = 337
broken: {'HA': (False, 14942241.0)}
cost : 1267
weight : 662



array([None, None], dtype=object)

In [ ]:
init = samplesets[0].samples()[-1]

## リバースアニーリング

In [ ]:
t = (0, 5, 20)
s = (1.0, 0.4, 1.0)

schedule = list(zip(t, s))

In [ ]:
plt.plot(t,s)
plt.show()

In [ ]:
sampleset = sampler.sample_qubo(
    Q, num_reads=30, anneal_schedule=schedule, initial_state=init)

In [ ]:
sampleset.record

In [ ]:
sample_info = transform_sample(sampleset)
output_sample_info(sample_info)